In [130]:
import numpy as np
from PIL import Image

In [131]:
data = "HHHHHHCCCCCCCAAAASSSSSFMMMMMMMMMEEIIIOP"
data1 = "1000001001000000111001101101101011000010111001001110100001000000111000001101000011011110110111001100101001011000010000001101100011010010110101101100101001000000111100101101111011101010111001000100000011011000110000101110000011101000110111101110000001000000110111101110010001000000110010001100101011100110110101101110100011011110111000000100000011000110110111101101101011100000111010101110100011001010111001000101100001000000110001101100001011011100"
data2 = "11010000001011100111"
data3 = "00101111110100011000"
original_data = 'LICENCE IRM A LA FST'
path = '../../random_binary_image.png'
img = np.array(Image.open(path))

with open("../../Textes/file2.txt", 'r') as fichier:
  text = fichier.read()

In [132]:
def compress_lz78(data):
  dictionary = {}
  encoded_data = []
  current_phrase = ""
  
  for char in data:
    current_phrase += char
    if current_phrase not in dictionary:
      if len(current_phrase) == 1:
        encoded_data.append((0, char))
      else:
        encoded_data.append((dictionary[current_phrase[:-1]], char))
    
      dictionary[current_phrase] = len(dictionary) + 1
      current_phrase = ""
    
  if current_phrase:
    encoded_data.append((dictionary[current_phrase], ""))
    
  return encoded_data

In [133]:
def decompress_lz78(compressed_data):
    dictionary = {0: ''}
    decoded_data = ''
    
    for index, char in compressed_data:
        if index in dictionary:
            phrase = dictionary[index] + char
        else:
            phrase = dictionary[index - 1] + char
        
        decoded_data += phrase
        dictionary[len(dictionary)] = phrase
    
    return decoded_data

In [134]:
compressed = compress_lz78(data1)

In [135]:
data1 == decompress_lz78(compressed)

True

In [136]:
def taux_compression(data, compressed_data):
    occurences = []
    
    for i, _ in compressed_data:
      occurences.append(i)
      
    maxi = max(occurences)
    bit_max = int(np.log2(maxi)) + 1
    taux = 1 - (len(compressed_data) * (8 + bit_max) / (len(data) * 8))
    return taux

In [137]:
def vecteur_colonne(matrice):
    shape = matrice.shape
    vecteur = np.ravel(matrice, order = 'F')
    return np.array(vecteur), shape

In [138]:
def inverse_vecteur_colonne(vecteur, shape):
    mat = np.array(vecteur).reshape((shape[0], shape[1]))
    return mat.T

In [139]:
vect, shape = vecteur_colonne(img)

In [140]:
(img == inverse_vecteur_colonne(vect, shape)).all()

True

In [141]:
def vecteur_to_binary(vecteur):
    if isinstance(vecteur, np.ndarray):
        # Si le vecteur est un tableau NumPy, on le convertit d'abord en une liste de booléens
        vecteur = vecteur.tolist()
    # Convertir chaque booléen en 0 ou 1
    binary_sequence = ''.join('1' if valeur else '0' for valeur in vecteur)
    return binary_sequence

In [142]:
def binary_to_vecteur(binary_sequence):
    # Convertir la séquence binaire en une liste de booléens
    vecteur = [True if bit == '1' else False for bit in binary_sequence]
    return np.array(vecteur)

In [143]:
vect1 = vecteur_to_binary(vect)

In [144]:
(vect == binary_to_vecteur(vect1)).all()

True

In [145]:
def image_to_lz78(image):
  data, shape = vecteur_ligne(image)
  binary_sequence = vecteur_to_binary(data)
  print(len(binary_sequence))
  encoded_text = compress_lz78(binary_sequence)
  return encoded_text, shape

In [146]:
def lz78_to_image(encoded, shape):
    decompressed = decompress_lz78(encoded)
    print(len(decompressed))
    original_vector = binary_to_vecteur(decompressed)
    original_image = inverse_vecteur_ligne(original_vector, shape)
    return original_image

In [147]:
encoded, shape = image_to_lz78(img)

NameError: name 'vecteur_ligne' is not defined

In [ ]:
decoded = lz78_to_image(encoded, shape)

62500


In [ ]:
taux_compression(vecteur_to_binary(img)[0], encoded)

-5259.0